In [ ]:
import json
import pickle
from tqdm import tqdm
import streamlit as st
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

DIR = "./test/"
model_path = './model_train_epoch_10.pth'
TEST_PKL = './test_paired_dict.pkl'
VAL_PKL = './val_paired_dict.pkl'
MLB_PKL = './mlb.pkl'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
THRESHOLD = 0.35

In [ ]:
def get_img_path(img_id):
    
    img_path = DIR + [*img_id][0] + '/' + [*img_id][1] + '/' + [*img_id][2] + '/' + [*img_id][3] + '/' + img_id
    return img_path

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
    
    def __len__(self):
        return len(self.data_dict)
    
    def __getitem__(self, idx):
        image_id, ingredients = [*self.data_dict.items()][idx]
        image_path = get_img_path(image_id)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        ingredients_tensor = torch.tensor(ingredients)
        return image, ingredients_tensor

In [ ]:
class ImgData(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        self.transform = transform
    
    def __len__(self):
        return 1
    
    def __getitem__(self, idx):
        image_path = self.img_path
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
num_unique_ingredients = 440

model = models.resnet50(pretrained=True)
num_in_features = model.fc.in_features
model.fc = nn.Linear(num_in_features, num_unique_ingredients)
model.load_state_dict(torch.load(model_path))

criterion = nn.BCEWithLogitsLoss()


model.to(device)

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
with open(TEST_PKL, 'rb') as file:    
    test_paired_dict = pickle.load(file)
with open(VAL_PKL, 'rb') as file:
    val_paired_dict = pickle.load(file)

with open(MLB_PKL, 'rb') as file:
    mlb = pickle.load(file)


test_set = MyDataset(test_paired_dict, transform=preprocess)
val_set = MyDataset(val_paired_dict, transform=preprocess)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [ ]:
def calc_scores(targets_idx, pred_idx):
    accuracy = []
    precision = []
    recall = []
    f1 = []
            
    for a,b in zip(targets_idx, pred_idx):
        targets_label = set(mlb.classes_[a])
        pred_label = set(mlb.classes_[b])
        intersection = targets_label.intersection(pred_label)
        pred_only = pred_label - intersection
        target_only = targets_label - intersection
        tp = len(intersection)
        fp = len(pred_only)
        fn = len(target_only)
        tn = num_unique_ingredients - (tp + fp + fn)
        accuracy.append((tp + tn) / (tp + fp + fn + tn))
        precision.append(tp / (tp + fp) if tp + fp != 0 else 0)
        recall.append(tp / (tp + fn) if tp + fn != 0 else 0)
        f1.append(2 * tp / (2 * tp + fp + fn) if tp + fp + fn != 0 else 0)
    return np.average(accuracy), np.average(precision), np.average(recall), np.average(f1)

In [ ]:
def val_test(data_set, batch_size=BATCH_SIZE, threshold = THRESHOLD):
        
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0

    acc_ls = []
    pre_ls = []
    rec_ls = []
    f1_ls = []
    val_loader = DataLoader(dataset=data_set, batch_size=batch_size, shuffle=False, drop_last=True)
    # Disable gradient calculation to speed up inference
    with torch.no_grad():
        
        with tqdm(val_loader, desc=f'Validation', unit='batch') as tqdm_loader:
            for val_data in tqdm_loader:
                val_inputs, val_labels = val_data
                
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device).float()
                

                val_outputs = model(val_inputs)
                
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item()
                
                val_outputs = torch.sigmoid(val_outputs)
                values = val_outputs.cpu().view(-1).numpy()
                
                
                plt.figure(figsize=(8, 6))
                plt.hist(values, bins=50, color='blue', alpha=0.7)
                plt.title('Distribution of Tensor Values')
                plt.xlabel('Value')
                plt.ylabel('Frequency')
                plt.grid(True)
                plt.savefig('./distribution.png')
                plt.show()
                

                
                targets_idx = [[idx for idx, col in enumerate(row) if col == 1] for row in val_labels.cpu()]
                pred_idx = [[idx for idx, col in enumerate(row) if col >= threshold] for row in val_outputs.cpu()]
                
                avg_accuracy, avg_precision, avg_recall, avg_f1 = calc_scores(targets_idx, pred_idx)
                acc_ls.append(avg_accuracy)
                pre_ls.append(avg_precision)
                rec_ls.append(avg_recall)
                f1_ls.append(avg_f1)
                
                mean=[0.485, 0.456, 0.406]
                std=[0.229, 0.224, 0.225]
                unnormalize = transforms.Normalize(mean=[-m/s for m, s in zip(mean, std)], std=[1/s for s in std])
                unnormalized_img_tensor = unnormalize(val_inputs)[0]
                unnormalized_tensor = torch.clamp(unnormalized_img_tensor, 0, 1)
                to_pil = transforms.ToPILImage()
                #image = to_pil(unnormalized_tensor) 
                #plt.axis('off')
                #plt.imshow(image)
                
                tqdm_loader.set_postfix(accuracy=avg_accuracy, precision=avg_precision, recall=avg_recall, f1=avg_f1)

    accuracy = np.average(acc_ls) 
    precision = np.average(pre_ls)
    recall = np.average(rec_ls)
    f1 = np.average(f1_ls)

    #print(f"Validation Loss: {val_loss}, Accuracy: {val_accuracy}%")
    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, Threshold: {threshold}")

In [ ]:
def test(img_path, batch_size=1, threshold =THRESHOLD):
    test_loader = DataLoader(ImgData(img_path, transform=preprocess), batch_size=1)
    for test_data in test_loader:
        test_inputs = test_data
        test_inputs = test_inputs.to(device)
        test_outputs = model(test_inputs)
        test_outputs = torch.sigmoid(test_outputs)
        targets_idx = [[idx for idx, col in enumerate(row) if col >= threshold] for row in test_outputs.cpu()]
        return [mlb.classes_[idx] for idx in targets_idx[0]]

In [ ]:
threshold_values = [0.1 + i * 0.05 for i in range(11)]
for thv in threshold_values:
    val_test(val_set, threshold=thv)

In [ ]:
threshold_values = [0.1 + i * 0.05 for i in range(11)]
for thv in threshold_values:
    val_test(test_set, threshold=thv)

In [ ]:
test_img = "./test_image.jpg"
labels = test(test_img, batch_size=BATCH_SIZE, threshold =THRESHOLD)
label_string = ", ".join(labels)
print(f"The food in the image contains: {label_string}")

In [ ]:
#on validation, batchsize = 16
thresholds = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
precision_scores = [0.32, 0.39, 0.42, 0.47, 0.52, 0.54, 0.56, 0.56, 0.55, 0.48, 0.28]
recall_scores = [0.65, 0.57, 0.50, 0.44, 0.40, 0.35, 0.31, 0.27, 0.22, 0.18, 0.06]
f1_scores = [0.41, 0.46, 0.45, 0.45, 0.42, 0.40, 0.37, 0.33, 0.29, 0.25, 0.10]

plt.figure(figsize=(10, 6))

plt.plot(thresholds, precision_scores, marker='o', label='Precision')
plt.plot(thresholds, recall_scores, marker='s', label='Recall')
plt.plot(thresholds, f1_scores, marker='d', label='F1 Score')

plt.title('Precision, Recall, and F1 Score vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.xticks(thresholds)
plt.savefig('./val_scores.png')

plt.show()


In [ ]:
#test, batchsize = 16
thresholds = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
precision = [0.316, 0.388, 0.446, 0.492, 0.526, 0.552, 0.567, 0.566, 0.552, 0.519, 0.466]
recall = [0.666, 0.589, 0.526, 0.472, 0.422, 0.375, 0.330, 0.284, 0.240, 0.195, 0.150]
f1_score = [0.413, 0.450, 0.462, 0.458, 0.443, 0.419, 0.389, 0.351, 0.310, 0.262, 0.211]

plt.figure(figsize=(10, 6))

plt.plot(thresholds, precision_scores, marker='o', label='Precision')
plt.plot(thresholds, recall_scores, marker='s', label='Recall')
plt.plot(thresholds, f1_scores, marker='d', label='F1 Score')

plt.title('Precision, Recall, and F1 Score vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.xticks(thresholds)
plt.savefig('./test_scores.png')

plt.show()